In [1]:
y_true = []
row = []
data = []

date = []
slot = []
dist = []

with open("training_data", 'r') as fr:
    while True:
        event = fr.readline().strip()
        if not event: break;
        
        event = event.split(' ')
        y_true.append(int(event[0]))
        date.append(int(event[1]))
        slot.append(int(event[2]))
        dist.append(int(event[3]))
        
        vidx = []
        vdata = []
        for itm in event[4:]:
            idx, val = itm.split(':')
            vidx.append(int(idx))
            vdata.append(float(val))
        row.append(vidx)
        data.append(vdata)

In [2]:
from extend_feature import *

In [3]:
for i, (r, d) in enumerate(zip(row, data)):
    r, d = extend_2dfeature(r, d)
    row[i]=r
    data[i]=d

In [4]:
col = []
for i in xrange(len(row)):
    col.append([i]*len(row[i]))

In [5]:
from scipy.sparse import coo_matrix
X = coo_matrix((np.hstack(data), (np.hstack(col), np.hstack(row))), shape=(len(y_true), ext[-1]))
X = X.tocsr()

In [6]:
X

<84693x20969 sparse matrix of type '<type 'numpy.float64'>'
	with 1207340 stored elements in Compressed Sparse Row format>

In [6]:
Y = np.asarray(y_true)

In [7]:
weight = np.zeros(Y.shape)
for i, y in enumerate(Y):
    if y==0:
        weight[i]=1
    else:
        weight[i]=1/y

In [20]:
y_true = []
col = []
data = []

date = []
slotv = []
distv = []

with open("validation_data", 'r') as fr:
    while True:
        event = fr.readline().strip()
        if not event: break;
        
        event = event.split(' ')
        y_true.append(int(event[0]))
        date.append(int(event[1]))
        slotv.append(int(event[2]))
        distv.append(int(event[3]))
        
        vidx = []
        vdata = []
        for itm in event[4:]:
            idx, val = itm.split(':')
            vidx.append(int(idx))
            vdata.append(float(val))
        col.append(vidx)
        data.append(vdata)

In [13]:
for i, (c, d) in enumerate(zip(col, data)):
    c, d = extend_2dfeature(c, d)
    col[i]=c
    data[i]=d

In [14]:
row = []
for i in xrange(len(col)):
    row.append([i]*len(col[i]))

In [15]:
Xv = coo_matrix((np.hstack(data), (np.hstack(row), np.hstack(col))), shape=(len(y_true), ext[-1]))
Xv = Xv.tocsr()
Yv = np.asarray(y_true)

In [8]:
from sklearn.linear_model import Ridge
lr = Ridge(alpha=10)
lr.fit(X, Y, weight)

Ridge(alpha=10, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [9]:
Y_pred = lr.predict(X)

In [14]:
Y_pred

array([ 8.37986386,  5.08294088,  5.6898639 , ..., -0.09732153,
       -0.43269772, -0.43170051])

In [10]:
Y_pred[Y_pred<0]=0

In [23]:
Yv

array([4, 7, 7, ..., 0, 1, 0])

In [16]:
Yv_pred = lr.predict(Xv)

In [17]:
Yv_pred[Yv_pred<0]=0

In [10]:
ext

[275, 18125, 20699, 20969]

In [1]:
def mape(y_true, y_pred, slot, dist):
    loss = np.zeros((144, 66))
    for i, (s, d) in enumerate(slot, dist):
        d -=1
        if y_true!=0:
            loss[s, d] = np.abs(1-y_pred[i]/y_true[i])
    l = np.sum(loss, axis=0)/144
    return np.sum(l)/66

In [16]:
loss = np.zeros((144, 66))

In [19]:
np.sum(loss, axis=0).shape

(66,)

In [21]:
mape(Yv, Yv_pred, slotv, distv)

0.24000297358666692

In [17]:
np.ones(4)/np.arange(1,5)

array([ 1.        ,  0.5       ,  0.33333333,  0.25      ])

In [1]:
record = []
with open("season_1/test_set_1/read_me_1.txt", 'r') as fr:
    while True:
        r = fr.readline().strip()
        if not r: break;
        
        for i in xrange(1,67):
            record.append((i, r))

In [3]:
len(record)

2838

In [4]:
col = []
data = []
with open("test_data") as fr:
    while True:
        r = fr.readline().strip()
        if not r: break;
        r = r.split(" ")
        
        vidx = []
        vdata = []
        for x in r:
            idx, val = x.split(":")
            vidx.append(int(idx))
            vdata.append(float(val))
        
        col.append(vidx)
        data.append(vdata)

In [6]:
from evaluation import *

In [8]:
X, Y, slot, dist = getdata("training_data_total")

In [9]:
from sklearn.linear_model import Ridge


weight = np.zeros(Y.shape)
for i, y in enumerate(Y):
    if y==0:
        weight[i]=1
    else:
        weight[i]=1/y

lr = Ridge(alpha=10)
lr.fit(X, Y, weight)

Ridge(alpha=10, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [10]:
Xt, recs = gettestdata("test_data", "season_1/test_set_1/read_me_1.txt")

In [17]:
Y_pred = lr.predict(X)

In [18]:
Y[Y<0]=0

In [14]:
Yt

array([ 0.50515934,  0.18966589,  0.0744843 , ...,  0.17523388,
        0.32034611,  0.13346256])

In [16]:
with open("result.csv", "w") as fw:
    for i, (r, s) in enumerate(recs):
        fw.write("{0},{1},{2}\n".format(r, s, Yt[i]))

In [19]:
mape(Y, Y_pred, slot, dist)

0.10702183890449453

In [3]:
X.shape

(56985, 278)

In [19]:
from evaluation import *

X, Y, slot_train, dist_train = getdata('training_data')

In [20]:
X1, Y1, slot1, dist1 = getdata("validation1_data")
X2, Y2, slot2, dist2 = getdata("validation2_data")

In [21]:
weight = np.zeros(Y.shape)
for i, y in enumerate(Y):
    if y==0:
        weight[i]=0
    else:
        weight[i]=1.0/y

In [25]:
from sklearn.linear_model import Ridge
lr = Ridge(alpha=40)
lr.fit(X, Y, weight)

Ridge(alpha=40, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [26]:
Y1_pred = lr.predict(X1)
Y2_pred = lr.predict(X2)
Y1_pred[Y1_pred<0]=0
Y2_pred[Y2_pred<0]=0
print (mape(Y1, Y1_pred, slot1, dist1)+mape(Y2, Y2_pred, slot2, dist2))/2

0.480073979685


In [27]:
Y_pred = lr.predict(X)
Y_pred[Y_pred<0]=0
mape(Y, Y_pred, slot_train, dist_train)

0.46387521799056297

In [8]:
Y_pred

array([ 2.15119564,  2.29900194,  2.0995028 , ...,  1.031052  ,
        0.83814329,  0.80688659])

In [8]:
from sklearn.kernel_ridge import KernelRidge

lr = KernelRidge(kernel='rbf', gamma=0.25)
lr.fit(X, Y, weight)

MemoryError: 

In [55]:
Y1_pred = lr.predict(X1)
Y2_pred = lr.predict(X2)
Y1_pred[Y1_pred<0]=0
Y2_pred[Y2_pred<0]=0
print (mape(Y1, Y1_pred, slot1, dist1)+mape(Y2, Y2_pred, slot2, dist2))/2

0.883015471721


In [56]:
Y_pred = lr.predict(X)
Y_pred[Y_pred<0]=0
mape(Y, Y_pred, slot_train, dist_train)

0.91671565220877183

In [20]:
Y1_pred = (lr.predict(X1)+rf.predict(X1))/2
Y2_pred = (lr.predict(X2)+rf.predict(X2))/2
Y1_pred[Y1_pred<0]=0
Y2_pred[Y2_pred<0]=0
print (mape(Y1, Y1_pred, slot1, dist1)+mape(Y2, Y2_pred, slot2, dist2))/2

0.472960543559


| Method description | Training set | Validation set |
|---|---|---|
| Rigle, alpha=50, weighted. 144slot+24h+weekday+district+pre7dstatus+weather | 0.5123620179427425 | 0.517201372498 |
| Rigle, alpha=50, weighted. 144slot+24h+weekday+district+pre7dstatus+pre1slotstatus+weather | 0.46628984499361303 | 0.486138200681 |
| RF, tree=50, min_samples_leaf=16, weighted. 144slot+24h+weekday+district+pre7dstatus+pre1slotstatus+weather | 0.44512307730615425 | 0.483293242609 |
| RF, tree=50, min_samples_leaf=20, weighted. 144slot+24h+weekday+district+pre7dstatus+pre1slotstatus+1storderstatus | 0.45543519259309279 | 0.48353572376 |
| Rigle, alpha=40, weighted. 144slot+24h+weekday+district+pre7dstatus+pre1slotstatus | 0.46387521799056297 | 0.480073979685 |